# **Install and Import Deoendencies**

In [ ]:
!pip install transformers datasets evaluate accelerate
from transformers import pipeline,AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from datasets import load_dataset
import numpy as np
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00


# **Demo Pipeline(before training)**

In [ ]:
demo_pipe = pipeline(task = "sentiment-analysis")

text = 'This movie was fantastic!'

result = demo_pipe(text)

print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998781681060791}]


In [ ]:
demo_pipeline = pipeline(task = 'sentiment-analysis')

text = 'The film was terrible and boring'

result = demo_pipeline(text)

print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


[{'label': 'NEGATIVE', 'score': 0.9998013377189636}]


# **Load IMDB Dataset(Binary_Sentiment)**

In [ ]:
dataset = load_dataset('imdb')
print(dataset)

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
print(dataset['train']) # show meta data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [ ]:
print(dataset['train'][10]) # show actual data 10th row

{'text': 'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is a

In [ ]:
print(dataset['train'].features) # show features

{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}


In [ ]:
dataset['train'][:10]

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [ ]:
print(dataset['test'][2])

{'text': "its a totally average film with a few semi-alright action sequences that make the plot seem a little better and remind the viewer of the classic van dam films. parts of the plot don't make sense and seem to be added in to use up time. the end plot is that of a very basic type that doesn't leave the viewer guessing and any twists are obvious from the beginning. the end scene with the flask backs don't make sense as they are added in and seem to have little relevance to the history of van dam's character. not really worth watching again, bit disappointed in the end production, even though it is apparent it was shot on a low budget certain shots and sections in the film are of poor directed quality", 'label': 0}


# *Tokenization*

In [ ]:
model = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model)

def tokenize_function(example):
  return tokenizer(example['text'],truncation = True,padding = 'max_length',max_length = 256)

tokenized = dataset.map(tokenize_function,batched = True)
small_train = tokenized['train'].shuffle(seed = 42).select(range(2000))
small_test = tokenized['test'].shuffle(seed = 42).select(range(1000))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

# **Fine Tunning Using Hugging Face Trainer**

In [ ]:
model_name = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 2)


args = TrainingArguments(
    output_dir = './binary_cls_results',
    eval_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs =2,
    weight_decay = 0.01,
    logging_dir = './binary_cls_logs'
)

accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')

def compute_metrics(eval_pred):
  logits,labels = eval_pred
  preds = np.argmax(logits,axis=-1)

  return {
      'accuracy':accuracy.compute(predictions = preds,references = labels)['accuracy'],
      'precision':precision.compute(predictions = preds,references = labels, average="binary")['precision'],
      'recall':recall.compute(predictions = preds,references = labels, average="binary")['recall'],
      'f1':f1.compute(predictions = preds,references = labels, average="binary")['f1']
  }

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = small_train,
    eval_dataset = small_test,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-1624883886.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: swapnilsurushe-ml (swapnilsurushe-ml-ab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.352506,0.843000,0.790861,0.922131,0.851466
2,No log,0.325780,0.873000,0.860279,0.883197,0.871587


TrainOutput(global_step=250, training_loss=0.366241455078125, metrics={'train_runtime': 126.7838, 'train_samples_per_second': 31.55, 'train_steps_per_second': 1.972, 'total_flos': 264934797312000.0, 'train_loss': 0.366241455078125, 'epoch': 2.0})

# **Evaluate Model**

In [ ]:
result = trainer.evaluate()
print(result)

{'eval_loss': 0.3257797062397003, 'eval_accuracy': 0.873, 'eval_precision': 0.8602794411177644, 'eval_recall': 0.8831967213114754, 'eval_f1': 0.871587462082912, 'eval_runtime': 7.1402, 'eval_samples_per_second': 140.053, 'eval_steps_per_second': 8.823, 'epoch': 2.0}


# **Save Model Locally**

In [ ]:
save_dir = './binary_text_classification_model'
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print('Saved model to',save_dir)

Saved model to ./binary_text_classification_model


# **Load the Saved Model & Tokenizer**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model = AutoModelForSequenceClassification.from_pretrained(save_dir)

# **Predictions**

In [ ]:
classifier = pipeline(
    task = 'sentiment-analysis',
    model = model,
    tokenizer = tokenizer
)

text = "I really loved this movie! It was fantastic."

result = classifier(text)
print(result)

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.9390419125556946}]


In [ ]:
# predicting multiple reviews

reviews = [
    "The movie was absolutely amazing!",
    "I hated the acting and the story was boring.",
    "It was okay, but not the best."
]

result = classifier(reviews)
print(result)

[{'label': 'LABEL_1', 'score': 0.9249285459518433}, {'label': 'LABEL_0', 'score': 0.93119877576828}, {'label': 'LABEL_0', 'score': 0.716031551361084}]


# **Push to Hugging Face Hub**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

trainer.push_to_hub('binary_text_classification_model_swapnil12')

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...inary_cls_results/model.safetensors:   0%|          |  574kB /  268MB            

  ...inary_cls_results/training_args.bin:   1%|          |  47.0B / 5.71kB            

CommitInfo(commit_url='https://huggingface.co/swapnil-12/binary_cls_results/commit/f1b6a4554c97e312044dcadcf2ff3921ca2739f9', commit_message='binary_text_classification_model_swapnil12', commit_description='', oid='f1b6a4554c97e312044dcadcf2ff3921ca2739f9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/swapnil-12/binary_cls_results', endpoint='https://huggingface.co', repo_type='model', repo_id='swapnil-12/binary_cls_results'), pr_revision=None, pr_num=None)

# **Using from Huggingface hub**

In [ ]:
sentiment_pipeline = pipeline(
    task = 'sentiment-analysis',
    model = "swapnil-12/binary_cls_results"
)

text = ' I love eating traditional and healthy food'

result = sentiment_pipeline(text)
print(result)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.8899939656257629}]
